# load libraries and data

In [1]:
from torch import nn
import torch
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
"""k fold 대신 stratified k fold"""
from sklearn.model_selection import StratifiedKFold
import torch
import itertools
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm
from pathlib import Path
import os

# psychosocial + rsfMRI + strurctural MRI + diffusion MRI


dataset_name = 'rsf_s_dMRI'
train_out = Path(os.getcwd()+'/Data/si_ppc_'+dataset_name+'_train.csv')
test_out =Path(os.getcwd()+'/Data/si_ppc_'+dataset_name+'_test.csv')


train_data= pd.read_csv(train_out)
test_data= pd.read_csv(test_out)


target ='Suicidalideation'
unused_feat = ['abcd_site','kfold']

# mri feature가 시작하는 column의 index 구하기
# np.where의 결과값이 array에 들어가기 때문에 방금 계산해 넣어놓은 [0]번째 값을 가져온다.
start_psycho_index = np.where(test_data.columns.values == "race.ethnicity_1")[0][0]
start_rsfmri_index = np.where(test_data.columns.values == "rsfmri_var_cort.destrieux_g.and.s.frontomargin.lh")[0][0]
# start_rsfmri_index = np.where(test_data.columns.values == "rsfmri_var_cort.destrieux_g.and.s.frontomargin.lh")[0][0]
start_structmri_index = np.where(test_data.columns.values == "lh_bankssts_area._.1")[0][0]
start_diffmri_index = np.where(test_data.columns.values == "con_L.BSTS_L.CACG_count")[0][0]

psychosocial = list(test_data.columns[start_psycho_index:start_rsfmri_index])
rsf_mri = list(test_data.columns[start_rsfmri_index:start_structmri_index])
structural_mri = list(test_data.columns[start_structmri_index:start_diffmri_index])
diffusion_mri = list(test_data.columns[start_diffmri_index:])



Num_FOLDS  = 5
# the number of feature that you want to show 
Num_feat = 20


print(len(train_data), len(train_data.columns))
print(len(psychosocial))
print(len(rsf_mri))
print(len(structural_mri))
print(len(diffusion_mri))

5510 4750
129
148
984
3485


# Fine object featrues

# For getting feature importance 

In [2]:
def feature(Num_feat, clf_res, test_data_processed, features):
    importance_res = []
    for i in clf_res:
        importance_clf =i.feature_importances_
        importance_res.append(importance_clf)
    
    importance=[importance_res[0][i]/5+importance_res[1][i]/5+ importance_res[2][i]/5+ importance_res[3][i]/5+ importance_res[4][i]/5 for i in range(len(importance_res[1]))]
    
    #print(importance)
    #plt.plot(importance)
    #plt.show()

    #importance_sort = importance.sort(reverse=True)
    
    #labels_importance=importance.argsort()[::-1]

    #feat_name_sort=test_data_processed[features].columns[labels_importance]
    feat_name_sort = test_data_processed[features].columns
    important_features = pd.DataFrame([importance],columns = feat_name_sort, index=['Importance']) 
    important_features =important_features.transpose().sort_values(by=['Importance'], ascending=False)
    important_features = important_features.head(50)
    """
    for i in range (len(test_data_processed[features].columns)):
        feature = pd.DataFrame([[feat_name_sort[i],importance_sort[i]]], columns = ['feature name', 'ratio'])
        important_features=pd.concat([important_features,feature])
    """
    return important_features

# Define function for preprocessing for Cross validation

In [3]:
def preprocessing (train_data, test_data, NUM_FOLDS):
    test_data_processed= test_data.fillna(0).reset_index(drop=True)
    train_data_processed = train_data.fillna(0).reset_index(drop=True)
    
    # 추가? 값 변경?
    test_data_processed["kfold"] = -1
    train_data_processed["kfold"] = -1

    # frac: 전체 row 중 몇 %를 반환할 지 결정 -> frac=1을 설정해서 모든 데이터를 반환
    # random_state: 추후 이것과 동일한 샘플링을 재현하기 위함
    # sample: 데이터에서 임의의 샘플 선정 -> frac=1이면 전체 data의 순서만 임의로 바뀜
    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)
    
    """stratified로 수정"""
    # kf = KFold(n_splits=NUM_FOLDS) 
    kf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state =0)
    
    # enumerate: 각 split된 data set 순서대로 index를 함께 반환
        # ex. fold 0 일 때, 80%는 trn_, 20%는 val_
    for fold, (trn_, val_) in enumerate(kf.split(X=train_data_processed, y=train_data_processed[target])):
        #print(len(trn_), len(val_)) -> 출력: 4408, 1102
        # 'kfold' 칼럼은 cross validation할 때 fold 순서를 지정해놓은 것. 
        train_data_processed.loc[val_, 'kfold'] = fold
    
    print("done preprocessing")
    return train_data_processed, test_data_processed

# Finding best parameters

In [4]:
# Augmented
import torch
import itertools
from sklearn.metrics import confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm.notebook import tqdm


def find_bestpar(fold, train_data_processed, test_data_processed, features):
    
    """test data 생성"""
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    """
    # define hyperparameter space : learning rate, 
    n_ = [4,8,16]                              # 
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4] # learning rate
    w_ = [0.01, 0.001, 0.0001]                 # weight decay
    g_ = [0.95, 0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20, 30]                         # scheduler params - step_size
    
    # Orginal hyperparameter space 
    """
    # define hyperparameter space (quick version)
    n_ = [4,16]
    lr_ = [2e-2,1e-3]
    w_ = [0.01,0.001]
    g_ = [0.95,0.99]
    ss_ = [10,30]
    
    
    
    
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        """===================Cross Validation==================="""
        
        """validation & test 결과"""
        valid_res = []
        test_auc_res = []
        test_acc_res = []
     
        for i in range(fold):
            #print("fold ", i)
            # 5개의 fold 사용했으므로 변수 fold 값은 차례대로 0,1,2,3,4 중 하나
            clf = TabNetClassifier(n_a = hy[0],
                                n_d = hy[0],
                                optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                                scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                verbose=0)
            
            df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
            df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당
            
            X_train = df_train[features].values
            Y_train = df_train[target].values
            
            X_valid = df_valid[features].values
            Y_valid = df_valid[target].values
            
            


            clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                    eval_name=['train', 'valid'], eval_metric=['auc'],
                    max_epochs=200 , patience=20)
       
            preds_acc = clf.predict(X_test)
            preds_prob = clf.predict_proba(X_test)
            test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
            test_acc = accuracy_score(preds_acc, Y_test)
            
            valid_res.append(clf.best_cost)
            test_auc_res.append(test_auc)
            test_acc_res.append(test_acc)
            print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f%%'%test_auc, 'Test ACC: %.3f%%'%test_acc)
    
        #print(valid_res)
        #print(test_auc_res)
        #print(test_acc_res)
        """valid와 test의 평균, 표준편차 출력"""
        print("=====parameter별 valid, test score=====")
        print("Validation 평균: %3f"%np.mean(valid_res), "Test AUC 평균: %3f"%np.mean(test_auc_res), "Test ACC 평균: %3f"%np.mean(test_acc_res))

        if np.mean(test_auc_res)>max_auc:
            print("Find new maximum AUC!!")
            max_hy = hy
            max_auc = np.mean(test_auc_res)
    
    return max_hy


# Train with best parameter

In [5]:
def bestpar_tuning(fold, train_data_processed, test_data_processed, max_hy, features):
    hy = max_hy
    print("Max hy:" ,hy)
    X_test = test_data_processed[features].values
    Y_test = test_data_processed[target].values
    """validation & test 결과"""
    valid_res = []
    test_auc_res = []
    test_acc_res = []
    clf_res = []
    preds_prob_res = []
    
    """해당 버전에 추가된 코드"""    
    y_valid_true = []
    y_valid_pred = []
    y_test_pred = []
    
    y_valid_subject = []
    y_test_subject = []
    
    
    for i in range(fold):
        clf = TabNetClassifier(n_a = hy[0],n_d = hy[0],
                           optimizer_params = dict(lr=hy[1], weight_decay=hy[2]),
                           scheduler_params={"step_size":hy[4], "gamma":hy[3]},
                           scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           verbose=0)
        
        # 5개의 fold 사용했으므로 변수 fold 값은 차례대로 0,1,2,3,4 중 하나
        df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
        df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당
            
        X_train = df_train[features].values
        Y_train = df_train[target].values
            
        X_valid = df_valid[features].values
        Y_valid = df_valid[target].values
        
        #print(X_valid)
        # 학습 전 subject key 가져옴
        y_valid_subject.append(df_valid['subjectkey'].values)
        y_test_subject.append(test_data_processed['subjectkey'].values)  
        """해당 버전 추가 코드"""
        y_valid_true.append(Y_valid)        
        
        # 학습
        clf.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                        eval_name=['train', 'valid'], eval_metric=['auc'],
                        max_epochs=200 , patience=20)
        
        # 결과들
        """fold별 validation & test data에 대한 target 예측 배열"""
        """해당 버전 추가 코드"""
        preds_prob_val= clf.predict_proba(X_valid)[:,1]
        y_valid_pred.append(preds_prob_val)
        # y_valid_pred.append(clf.predict(X_valid))
        y_test_pred.append(clf.predict(X_test))


        preds_acc = clf.predict(X_test)
        preds_prob = clf.predict_proba(X_test)
        test_auc = roc_auc_score(y_score=preds_prob[:,1], y_true=Y_test)
        test_acc = accuracy_score(preds_acc, Y_test)

                    
        valid_res.append(clf.best_cost)
        test_auc_res.append(test_auc)
        test_acc_res.append(test_acc)
        print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% clf.best_cost, 'Test AUC: %.3f'%test_auc, 'Test ACC: %.3f'%test_acc)
        preds_prob_res.append(preds_prob)
        clf_res.append(clf)
    
    """valid와 test의 평균, 표준편차 출력"""
    print("Validation 평균: %3f"%np.mean(valid_res), "Test AUC 평균: %3f "%np.mean(test_auc_res), "Test ACC 평균: %3f"%np.mean(test_acc_res))
    
    preds_prob=[preds_prob_res[0][i]/5+preds_prob_res[1][i]/5+ preds_prob_res[2][i]/5+ preds_prob_res[3][i]/5+ preds_prob_res[4][i]/5 for i in range(len(preds_prob_res[1]))]
    preds_prob = np.array(preds_prob)
    valid_result = np.mean(valid_res)
    test_auc = np.mean(test_auc_res)
    test_acc = np.mean(test_acc_res) 
    
    #return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject
    """수정 필요"""
    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true



# Run function, Split data and and cross validation. This needs to be modified

In [6]:
def run(train_data_processed, test_data_processed, fold, Num_feat, features):
    name_test = test_data_processed['subjectkey'].values
    print("-------------------------------Training Begining-------------------------------")
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    g_ = [0.95, 0.99, 0.9]
    ss_ = [10, 20, 30]
    all_ = [n_, lr_, w_, g_, ss_]
    h_space = [s for s in itertools.product(*all_)]
    
    # Start training
    max_hy = find_bestpar(fold, train_data_processed, test_data_processed, features)
    
    # if you want to just test the code, you should use this
    #max_hy = h_space[0]
    #print("Found maximum hyperparmeter, now work with that")
    
    print("-------------------------------Testing Begining-------------------------------")
    test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true= bestpar_tuning(fold, 
                                                                                        train_data_processed, 
                                                                                        test_data_processed, 
                                                                                        max_hy, 
                                                                                        features)
    
    #print("-------------------------------Important Feature-------------------------------")
    import_feat=feature(Num_feat, clf_res, test_data_processed, features)
    #import_feat=0
    #preds_val_prob = clf.predict_proba(X_valid)

    return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true


In [7]:
"""해당 버전 추가 코드"""
def save_prob_with_true(model):
    modeltype= "" 
    combined_model=pd.DataFrame({f"subjectkey": model.y_test_subject[0], f"Y_{modeltype}":model.Y_test, f"preds_prob_{modeltype}" :model.preds_prob[:,1]} )
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    #mdoel.import_feat_every.to_csv(f"{modeltype}_features.csv")

    return combined_model
def save_prob_with_true_valid(model):
    modeltype= "" 
    
    combined_model=pd.DataFrame({f"subjectkey": list(itertools.chain(*model.y_valid_subject)), f"Y_{modeltype}":list(itertools.chain(*model.y_valid_true)), f"preds_prob_{modeltype}" :list(itertools.chain(*model.y_valid_pred) )})
    combined_model.to_csv(f"combined_forROC_{modeltype}_.csv")
    #model.import_feat_every.to_csv(f"{modeltype}_features.csv")

    return combined_model

# Main code 

In [8]:
class model():
    def __init__(self, train_data_processed, test_data_processed, Num_FOLDS, Num_feat, features):
        test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, import_feat, name_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject, y_valid_true = run(train_data_processed,
                                                                              test_data_processed,
                                                                              Num_FOLDS, 
                                                                              Num_feat, 
                                                                              features)
    
    
        self.train_data_processed = train_data_processed
        self.test_auc = test_auc
        self.test_acc = test_acc
        self.valid_result = valid_result
        self.clf_res = clf_res 
        self.preds_prob = preds_prob 
        self.X_test = X_test
        self.Y_test = Y_test
        self.import_feat =  import_feat
        self.name_test = name_test
        self.features = features
        self.y_valid_pred =y_valid_pred
        self.y_test_pred = y_test_pred
        self.y_valid_subject= y_valid_subject
        self.y_test_subject = y_test_subject
        self.y_valid_true = y_valid_true
        """해당 버전 추가 코드"""        
        test_prob_result = save_prob_with_true(self)
        valid_prob_result = save_prob_with_true_valid(self)

        """해당 버전 추가 코드"""
        self.test_prob_result= test_prob_result
        self.valid_prob_result = valid_prob_result        

In [10]:
train_data_processed, test_data_processed = preprocessing (train_data, test_data, Num_FOLDS)


done preprocessing


In [ ]:
import dill

#path to dmri 
with open ("./Data/dMRI.pkl", 'rb') as file:
    diffusion_mri = dill.load(file)

new_features_D = [col for col in train_data_processed.columns if col in psychosocial + diffusion_mri.import_feat.index.tolist()]

new_DMRI = model(train_data_processed, test_data_processed, Num_FOLDS, Num_feat, new_features_D)

In [18]:
#save the model 

exp_type = "Newway_SI"

with open(f'./{exp_type}/new_DMRI.pkl', 'wb') as f:
    dill.dump(new_DMRI, f)

# For drawing ROC curve graph

In [ ]:
def rocvis(true , prob , label ) :
    from sklearn.metrics import roc_curve
    if type(true[0]) == str :
        from sklearn.preprocessing import LabelEncoder
        le = LabelEncoder()
        true = le.fit_transform(true)
    else :
        pass
    fpr, tpr, thresholds = roc_curve(true, prob)
    plt.plot(fpr, tpr, marker='.', label = label  )

# Draw ROC curve

In [ ]:
fig , ax = plt.subplots(figsize= (20,10))
plt.plot([0, 1], [0, 1], linestyle='--')
rocvis(new_DMRI.Y_test , new_DMRI.preds_prob[:,1] , f"new_DMRI:{round(new_DMRI.test_auc*100, 2)}%")


dataset_name="dmri"

plt.legend(fontsize = 40)
plt.title(dataset_name, fontsize= 50)
plt.xlabel("FP rate", fontsize =30)
plt.ylabel("TP rate", fontsize =30)
plt.xticks(size = 30)
plt.yticks(size = 30)

plt.tight_layout()
plt.savefig(f'{dataset_name}_ROC.png')

# Draw AUC per epochs and loss per epochs

feature_names=["Combined", "psychosocial", "MRI"]
clfs = [clf_all, clf_pheno, clf_mri ]
simple_name = "enbackfMRI"

def error_plot(clf):
    plt.plot(clf.history['train_auc'])
    plt.plot(clf.history['valid_auc'])

for i in range(3):
    error_plot(clfs[i])

plt.legend([f'train_{feature_names[0]}', f'valid_{feature_names[0]}', 
           f'train_{feature_names[1]}', f'valid_{feature_names[1]}',
           f'train_{feature_names[2]}', f'valid_{feature_names[2]}'])
plt.title(f'{simple_name}')
plt.xlabel('epochs')
plt.ylabel('AUC')
plt.tight_layout()

plt.savefig(f'AUC_per_epochs {simple_name}.png')

plt.show()          


feature_names=["combined", "psychosocial", "MRI"]
clfs = [clf_all, clf_pheno, clf_mri ]

def error_plot(clf):
    plt.plot(clf.history['loss'])

for i in range(3):
    error_plot(clfs[i])

plt.legend([f'{feature_names[0]}', 
           f'{feature_names[1]}',
           f'{feature_names[2]}'])
plt.title(f'{simple_name}')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.tight_layout()

plt.savefig(f'loss_per_epochs {simple_name}.png')

plt.show()